Data set link : https://drive.google.com/file/d/1CEql-OEexf9p02M5vCC1RDLXibHYE9Xz/view

In [18]:
# importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [20]:
# loading and exploring data
from google.colab import files
uploaded = files.upload()


Saving heart_disease_data.csv to heart_disease_data (2).csv


In [21]:
df = pd.read_csv('heart_disease_data.csv')

In [22]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [24]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [25]:
# checking the distribution of Target Variable
df['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

In [26]:
# Data Preprocessing
X = df.drop(columns=['target'], axis=1)
Y = df['target']


In [27]:
# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [28]:
# Splitting the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, stratify=Y, random_state=42)

In [29]:
# Class Balancing with SMOTE (Synthetic Minority Over-sampling Technique)
sm = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = sm.fit_resample(X_train, Y_train)


In [30]:
# Model Selection: Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [31]:
# Hyperparameter Tuning (example: grid search for hyperparameters)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [33]:
# Cross-Validation to evaluate the model's performance
from sklearn.model_selection import GridSearchCV
import time

# Set a maximum time limit in seconds (e.g., 3600 seconds = 1 hour)
max_time_limit = 3600
start_time = time.time()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

try:
    grid_search.fit(X_train_resampled, Y_train_resampled)
except KeyboardInterrupt:
    print("Grid search interrupted.")

# Check if the elapsed time exceeds the maximum time limit
elapsed_time = time.time() - start_time
if elapsed_time > max_time_limit:
    print(f"Grid search exceeded the maximum time limit of {max_time_limit} seconds.")
else:
    print("Grid search completed successfully.")

# Get the best parameters and results after the grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score}")


Grid search completed successfully.
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Score: 0.856168359941945


In [34]:
best_model = grid_search.best_estimator_

In [35]:
# Evaluate the model
Y_train_prediction = best_model.predict(X_train_resampled)
training_data_accuracy = accuracy_score(Y_train_resampled, Y_train_prediction)

In [36]:
Y_test_prediction = best_model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, Y_test_prediction)

In [37]:
print('Accuracy on Training data:', training_data_accuracy)
print('Accuracy on Test data:', test_data_accuracy)

Accuracy on Training data: 1.0
Accuracy on Test data: 0.819672131147541


In [38]:
# Classification Report for detailed evaluation
print('\nClassification Report on Test data:\n', classification_report(Y_test, Y_test_prediction))


Classification Report on Test data:
               precision    recall  f1-score   support

           0       0.90      0.68      0.78        28
           1       0.78      0.94      0.85        33

    accuracy                           0.82        61
   macro avg       0.84      0.81      0.81        61
weighted avg       0.83      0.82      0.82        61



In [39]:
# Building a Predictive System
input_data = np.array([62, 0, 0, 140, 268, 0, 0, 160, 0, 3.6, 0, 2, 2]).reshape(1, -1)
input_data_scaled = scaler.transform(input_data)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [40]:
prediction = best_model.predict(input_data_scaled)

In [ ]:
if prediction[0] == 0:
    print('The Person does not have a Heart Disease')
else:
    print('The Person has Heart Disease')

The Person does not have a Heart Disease
